In [10]:
'''
This code will grab USGS streamflow data for a defined period of interest 

'''

# first import the functions for downloading data from NWIS
import dataretrieval.nwis as nwis
import pandas as pd

# specify the USGS site code for which we want data.
site = ['01054500']

# get instantaneous values (iv)
df = nwis.get_record(sites=site, service='iv', start='2017-12-01', end='2018-01-01')
df.reset_index(inplace=True) #reset index to grab station date
df_quarter = df.iloc[:,[0,4]] #locates every row by the columns we want (date and flow)
df_quarter.columns = ['Date', 'Flow'] # rename column headers 
print(df_quarter)

## Make copy od dataframe to average flow every hour 
df_copy = df.copy()
df_copy['datetime'] = pd.to_datetime(df_copy['datetime']) #convert datetime to average every hour 
#df_copy.reset_index(inplace=True) #reset indexes so can grab "Date" column
#df_copy.drop("datetime", axis=1, inplace=True) #drop unneccsary date column now 
df_copy.index = df_copy['datetime'] # index so can pull date time in resample
df_avg = df_copy.resample('H').mean() # Average every hour based on datetime
df_avg.reset_index(inplace=True) #reset index again to have datetime 
df_avgflow = df_avg.iloc[:,[0,2]] #locates every row by the columns we want (date and flow)
df_avgflow.columns = ['Date', 'Flow']
print(df_avgflow)



                          Date  Flow
0    2017-12-01 00:00:00-05:00  4.39
1    2017-12-01 00:15:00-05:00  4.38
2    2017-12-01 00:30:00-05:00  4.38
3    2017-12-01 00:45:00-05:00  4.38
4    2017-12-01 01:00:00-05:00  4.38
...                        ...   ...
3067 2018-01-01 22:45:00-05:00  4.12
3068 2018-01-01 23:00:00-05:00  4.13
3069 2018-01-01 23:15:00-05:00  4.12
3070 2018-01-01 23:30:00-05:00  4.13
3071 2018-01-01 23:45:00-05:00  4.13

[3072 rows x 2 columns]
                     datetime   00060   00065
0   2017-12-01 00:00:00-05:00  3355.0  4.3825
1   2017-12-01 01:00:00-05:00  3350.0  4.3800
2   2017-12-01 02:00:00-05:00  3355.0  4.3825
3   2017-12-01 03:00:00-05:00  3350.0  4.3800
4   2017-12-01 04:00:00-05:00  3355.0  4.3825
..                        ...     ...     ...
763 2018-01-01 19:00:00-05:00  2760.0  4.1000
764 2018-01-01 20:00:00-05:00  2785.0  4.1125
765 2018-01-01 21:00:00-05:00  2790.0  4.1150
766 2018-01-01 22:00:00-05:00  2800.0  4.1200
767 2018-01-01 23:00:00-0